In [1]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/datasetcltv/flo_data_20k.csv


**CLTV Prediction**

**Business Problem**

Roadmap for FLO sales and marketing activities wants to determine. The company's medium-long-term plan existing customers to the company in the future so that they can estimating the potential value they will provide required.

**Dataset Summary**

* *master_id*: Unique customer number.
* *order_channel*: The channel that the purchase was made. (Android, iOS, Desktop, Mobile)
* *last_order_channel*: The channel that the last purchase was made.
* *first_order_date*: First purchase date of the customer.
* *last_order_date*: Last purchase date of the customer.
* *last_order_date_online*: Last online purchase date of the customer.
* *last_order_date_offline*: Last offline purchase date of the customer.
* *order_num_total_ever_online*: Total online purchase count of the customer.
* *order_num_total_ever_offline*: Total offline purchase count of the customer.
* *customer_value_total_ever_offline*: Total value spent on offline purchases by the customer.
* *customer_value_total_ever_online*: Total value spent on online purchases by the customer.
* *interested_in_categories_12*: The categories that customer purchased from in the last 12 months.

In [2]:
pip install lifetimes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import datetime as dt
import pandas as pd
import lifetimes
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 225)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [4]:
df_ = pd.read_csv("/kaggle/input/datasetcltv/flo_data_20k.csv")

In [5]:
df = df_.copy()

In [6]:
def general_info (dataframe):
    print("- Shape -")
    print(dataframe.shape,"\n\n")
    print("- Head -")
    print(dataframe.head(10),"\n\n")
    print("- Variables -")
    print(dataframe.columns,"\n\n")
    print("- Statistics -")
    print(dataframe.describe().T,"\n\n")
    print("- Null Value -")
    print(dataframe.isnull().sum(),"\n\n")
    print("- Variable Types-")
    print(dataframe.info(),"\n\n")

In [7]:
general_info(df)

- Shape -
(19945, 12) 


- Head -
                              master_id order_channel last_order_channel first_order_date last_order_date last_order_date_online last_order_date_offline  order_num_total_ever_online  order_num_total_ever_offline  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline       2020-10-30      2021-02-26             2021-02-21              2021-02-26                       4.0000                        1.0000   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile       2017-02-08      2021-02-16             2021-02-16              2020-01-10                      19.0000                        2.0000   
2  69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App       2019-11-27      2020-11-27             2020-11-27              2019-12-01                       3.0000                        2.0000   
3  1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App       2021-01-06      2021-01-17 

In [8]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)
    
def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        print(f"{col_name}\nLower limit = {low_limit}\nUpper limit = {up_limit}")
        return True
    else:
        print(f"{col_name}\nLower limit = {low_limit}\nUpper limit = {up_limit}")
        return False

In [9]:
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [10]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,order_num_total,customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800


In [11]:
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

df[date_columns].dtypes

first_order_date           datetime64[ns]
last_order_date            datetime64[ns]
last_order_date_online     datetime64[ns]
last_order_date_offline    datetime64[ns]
dtype: object

In [12]:
print(df["last_order_date"].max())
analysis_date = dt.datetime(2021,6,1) 

2021-05-30 00:00:00


In [13]:
# CLTV dataframe is formed by customer ids, weekly recency, weekly tenure, frequency and average monetary.
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]

# Recency is the date difference betwwen first and last orders of a customer
cltv_df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]) / dt.timedelta(weeks=1)

# Tenure is the date difference betwwen first order of a customer and analysis date.
cltv_df["T_weekly"] = (analysis_date - df["first_order_date"]) / dt.timedelta(weeks=1)

cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900


In [14]:
# BG/NBD model is prepared and parameters are obtained.
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 71.57, b: 0.26, r: 3.48>

In [15]:
#3 month interval.
cltv_df["exp_sales_3_month"] = bgf.predict(4*3,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9960
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9909
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6763
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7111
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3936


In [16]:
#6 month interval.
cltv_df["exp_sales_6_month"] = bgf.predict(4*6,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9960,1.9920
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9909,1.9818
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6763,1.3525
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7111,1.4222
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3936,0.7872


In [17]:
# Gamma-Gamma model is fitted and average values of customers
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'],
        cltv_df['monetary_cltv_avg'])

cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                       cltv_df['monetary_cltv_avg'])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9960,1.9920,193.6328
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9909,1.9818,96.6651
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6763,1.3525,120.9677
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7111,1.4222,67.3201
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3936,0.7872,114.3252


In [18]:
# CLTV values for 6 months 
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6, # in months
                                   freq="W", # Frequency type of T
                                   discount_rate=0.01)
cltv_df["cltv"] = cltv

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9960,1.9920,193.6328,404.6941
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9909,1.9818,96.6651,201.0020
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6763,1.3525,120.9677,171.6671
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7111,1.4222,67.3201,100.4584
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3936,0.7872,114.3252,94.4271


In [19]:
#top 20 CLTV values are presented.
cltv_df.sort_values("cltv",ascending=False)[:20]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,7.8571,4.0000,5176.5850,1.1297,2.2595,5348.2785,12679.1281
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.1429,13.1429,11.0000,2838.8555,2.0508,4.1017,2872.5603,12362.1890
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.1429,147.1429,68.0000,541.4454,3.9215,7.8430,542.5238,4464.4582
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.7143,67.2857,70.0000,263.4796,6.3500,12.6999,264.0193,3518.0635
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,394.0000,398.5714,202.0000,227.2530,5.2447,10.4893,227.4169,2502.8638
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.2857,74.5714,16.0000,622.0819,1.5994,3.1987,627.3416,2105.4665
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,383.0000,385.0000,82.0000,413.6354,2.2466,4.4932,414.3301,1953.3091
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,102.5714,131.4286,20.0000,605.1575,1.3878,2.7755,609.2498,1774.2302
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.0000,7.0000,1106.4671,0.7285,1.4569,1127.6121,1723.7007
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.4286,34.4286,8.0000,601.2262,1.2991,2.5981,611.4929,1666.9326


In [20]:
# Customers are segmented to 4 groups according to their 6 month CLTV values
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9960,1.9920,193.6328,404.6941,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9909,1.9818,96.6651,201.0020,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6763,1.3525,120.9677,171.6671,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7111,1.4222,67.3201,100.4584,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3936,0.7872,114.3252,94.4271,D


In [21]:
# The mean values of recency, frequency and monetary are observed for each segment.
cltv_df.groupby("cltv_segment").agg(["max","mean","count"]).reset_index()

cltv_segment recency_cltv_weekly                T_weekly                frequency              monetary_cltv_avg                exp_sales_3_month              exp_sales_6_month              exp_average_value           \
                               max     mean count      max     mean count       max   mean count               max     mean count               max   mean count               max   mean count               max     mean   
0            D            432.5714 139.3576  4987 436.2857 162.5802  4987  111.0000 3.8023  4987          284.9850  93.4656  4987            1.0811 0.4063  4987            2.1622 0.8127  4987          306.5914  99.0254   
1            C            433.0000  92.6934  4986 434.7143 112.9093  4986   47.0000 4.4049  4986          366.9967 126.0968  4986            1.5994 0.5258  4986            3.1988 1.0516  4986          384.7255 132.5840   
2            B            415.5714  81.7196  4986 423.5714 100.0417  4986   51.0000 5.0886  4986          539.9850 160.4783  4986            1.8225 0.6050  4986            3.6449 1.2100  4986          578.9761 167.8307   
3            A            433.4286  67.2745  4986 437.1429  82.3464  4986  202.0000 6.8034  4986         5176.5850 229.5689  4986            6.3500 0.7911  4986           12.6999 1.5822  4986         5348.2785 238.7558   

              cltv                 
  count        max     mean count  
0  4987   112.2440  80.0061  4987  
1  4986   165.9936 138.4894  4986  
2  4986   241.7165 200.4117  4986  
3  4986 12679.1281 372.5150  4986

Segment A should be treated as primary objectives to maintain and increase their value to the store. The C and D segments have little value to the company, these groups can be combined and analyzed as one.